In [1]:
from IPython.display import clear_output
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, roc_curve, auc, plot_roc_curve, roc_auc_score, classification_report
from sklearn.utils import resample
import numpy as np

In [2]:
#load train file
file_train = 'D:\Downloads\AI in Genomics - District 3\project\code\phyla_all_4514x1183_MAD_85p.csv'
df_train = pd.read_csv(file_train)

In [3]:
df_train

,Unnamed: 0,D_0__Archaea;D_1__Euryarchaeota;D_2__Halobacteria;D_3__Halobacteriales;D_4__Halococcaceae;__,D_0__Archaea;D_1__Euryarchaeota;D_2__Methanobacteria;D_3__Methanobacteriales;D_4__Methanobacteriaceae;D_5__Methanobacterium,D_0__Archaea;D_1__Euryarchaeota;D_2__Methanobacteria;D_3__Methanobacteriales;D_4__Methanobacteriaceae;D_5__Methanobrevibacter,D_0__Archaea;D_1__Euryarchaeota;D_2__Methanobacteria;D_3__Methanobacteriales;D_4__Methanobacteriaceae;D_5__Methanosphaera,D_0__Archaea;D_1__Euryarchaeota;D_2__Methanomicrobia;__;__;__,D_0__Archaea;D_1__Euryarchaeota;D_2__Thermoplasmata;D_3__Methanomassiliicoccales;D_4__Methanomassiliicoccaceae;D_5__Methanomassiliicoccus,D_0__Archaea;D_1__Euryarchaeota;D_2__Thermoplasmata;D_3__Methanomassiliicoccales;D_4__Methanomethylophilaceae;D_5__Candidatus Methanomethylophilus,D_0__Archaea;D_1__Euryarchaeota;D_2__Thermoplasmata;D_3__Methanomassiliicoccales;D_4__Methanomethylophilaceae;D_5__uncultured,D_0__Archaea;D_1__Euryarchaeota;D_2__Thermoplasmata;D_3__Methanomassiliicoccales;D_4__Methanomethylophilaceae;__,...,D_0__Bacteria;__;__;__;__;__,D_0__Eukaryota;D_1__Excavata;D_2__Discoba;D_3__Discicristata;D_6__Aphagea;D_7__Distigma,D_0__Eukaryota;__;__;__;__;__,Unassigned;__;__;__;__;__,col_site,diagnosis,sample_title,stool_biopsy,studyID,uc_cd
0,1939.100003,-0.448276,-0.448276,-0.555556,-0.465517,-0.448276,-0.467049,-0.448276,-0.444126,-0.451149,...,-0.866962,-0.446991,-0.446991,-0.446991,OSCCAR,1,1939.100003,stool,GEVERSM,CD
1,1939.100009,1.051724,1.051724,0.894444,1.034483,1.051724,1.028653,1.051724,1.051576,1.048851,...,0.290466,1.048711,1.048711,1.048711,OSCCAR,1,1939.100009,stool,GEVERSM,UC
2,1939.100015,-4.916667,-4.916667,178.941667,-4.933908,-4.916667,-4.922636,-4.916667,-4.899713,-4.919540,...,50.782705,-4.902579,-4.902579,-4.902579,OSCCAR,1,1939.100015,stool,GEVERSM,CD
3,1939.100016,1.014368,1.014368,0.858333,0.997126,1.014368,0.991404,1.014368,1.014327,1.011494,...,0.261641,1.011461,1.011461,1.011461,OSCCAR,1,1939.100016,stool,GEVERSM,UC
4,1939.100043,-0.586207,-0.586207,-0.688889,-0.603448,-0.586207,-0.604585,-0.586207,-0.581662,-0.589080,...,-0.973392,-0.584527,-0.584527,-0.584527,OSCCAR,1,1939.100043,stool,GEVERSM,UC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4509,ERR1073293,-1.551724,-1.551724,-1.622222,-1.568966,-1.551724,-1.567335,-1.551724,-1.544413,-1.554598,...,-1.718404,-1.547278,-1.547278,-1.547278,AG,0,10317.000004737,stool,AG,Control
4510,ERR1072841,-0.479885,-0.479885,-0.586111,-0.497126,-0.479885,-0.498567,-0.479885,-0.475645,-0.482759,...,29.844789,-0.478510,-0.478510,-0.478510,AG,0,10317.000001870,stool,AG,Control
4511,ERR1078202,-0.649425,-0.649425,-0.750000,-0.666667,-0.649425,-0.667622,-0.649425,-0.644699,-0.652299,...,-1.022173,-0.647564,-0.647564,-0.647564,AG,0,10317.000013111,stool,AG,Control
4512,ERR1075749,-0.488506,-0.488506,-0.594444,-0.505747,-0.488506,-0.507163,-0.488506,-0.484241,-0.491379,...,-0.898004,-0.487106,-0.487106,-0.487106,AG,0,10317.000013005,stool,AG,Control


In [4]:
#Train data
X_train = df_train[df_train.columns[1:-6]]
X_train = X_train.astype('float64')

#define y
y_train = df_train.diagnosis
print('Fraction of Infected: ', sum(y_train)/len(y_train))
print(X_train.shape)
print(y_train.shape)

Fraction of Infected:  0.4798404962339389
(4514, 1177)
(4514,)


In [5]:
#Hyperparameter optimization - SVM Linear - Fitted on phyla_all_4514x1183_MAD_85p dataset
C_range = [0.0001, 0.001, 0.01] #0.1, 1, 10 took forever
param_dict = {"C":C_range}

#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

#define model
model = SVC(kernel='linear')

scoring = ['accuracy', 'f1']
#grid search CV
grid_search = GridSearchCV(model, param_grid=param_dict, cv=5, refit=False, scoring=scoring, verbose=10, n_jobs=-1)
m_f = grid_search.fit(X_train, y_train)

print('Hyperparameter optimization on phyla_all_4514x1183_MAD_85p dataset SVM Linear')
d = pd.DataFrame(grid_search.cv_results_)
print('Best Accuracy')
print(d[d["rank_test_accuracy"] == 1][['params', "mean_test_accuracy"]], '\n')
print('Best f1')
print(d[d["rank_test_f1"] == 1][['params', "mean_test_f1"]], '\n')

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Hyperparameter optimization on phyla_all_4514x1183_MAD_85p dataset SVM Linear
Best Accuracy
          params  mean_test_accuracy
0  {'C': 0.0001}            0.837863 

Best f1
          params  mean_test_f1
0  {'C': 0.0001}      0.841371
None 

Best Precision


KeyError: 'rank_test_precision'

In [7]:
#Hyperparameter optimization - RF - Fitted on phyla_all_4514x1183_MAD_85p dataset

max_depth = list(range(1,11))
min_samples_leaf = [0.01,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]
param_dict = {"max_depth":max_depth, 'min_samples_leaf': min_samples_leaf}
#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)
model = RandomForestClassifier(n_estimators=100, criterion='gini', min_samples_split=2, 
                                 max_features='sqrt', max_leaf_nodes=None)

scoring = ['accuracy', 'f1']
#grid search CV
grid_search = GridSearchCV(model, param_grid=param_dict, cv=5, refit=False, scoring=scoring, verbose=10, n_jobs=-1)
m_f = grid_search.fit(X_train, y_train)

print('Hyperparameter optimization on phyla_all_4514x1183_MAD_85p dataset Random Forest')
d = pd.DataFrame(grid_search.cv_results_)
print('Best Accuracy')
print(d[d["rank_test_accuracy"] == 1][['params', "mean_test_accuracy"]], '\n')
print('Best f1')
print(d[d["rank_test_f1"] == 1][['params', "mean_test_f1"]], '\n')

Fitting 5 folds for each of 110 candidates, totalling 550 fits
Hyperparameter optimization on phyla_all_4514x1183_MAD_85p dataset Random Forest
Best Accuracy
                                        params  mean_test_accuracy
77  {'max_depth': 8, 'min_samples_leaf': 0.01}            0.863775 

Best f1
                                        params  mean_test_f1
77  {'max_depth': 8, 'min_samples_leaf': 0.01}      0.873355
None 



In [8]:
#Hyperparameter optimization - SVM RBF - Fitted on phyla_all_4514x1183_MAD_85p dataset

C_range = [0.0001, 0.001, 0.01, 1, 10]
gamma_range = [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2]

param_dict = {"C":C_range, 'gamma': gamma_range}
#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

model = SVC(kernel='rbf')

scoring = ['accuracy', 'f1']
#grid search CV
grid_search = GridSearchCV(model, param_grid=param_dict, cv=5, refit=False, scoring=scoring, verbose=10, n_jobs=-1)
m_f = grid_search.fit(X_train, y_train)

print('Hyperparameter optimization on phyla_all_4514x1183_MAD_85p dataset SVM RBF')
d = pd.DataFrame(grid_search.cv_results_)
print('Best Accuracy')
print(d[d["rank_test_accuracy"] == 1][['params', "mean_test_accuracy"]], '\n')
print('Best f1')
print(d[d["rank_test_f1"] == 1][['params', "mean_test_f1"]], '\n')

Fitting 5 folds for each of 65 candidates, totalling 325 fits
Hyperparameter optimization on phyla_all_4514x1183_MAD_85p dataset SVM RBF
Best Accuracy
                      params  mean_test_accuracy
42  {'C': 1, 'gamma': 1e-07}            0.877735 

Best f1
                      params  mean_test_f1
42  {'C': 1, 'gamma': 1e-07}      0.890507
None 

